<a href="https://colab.research.google.com/github/gibranfp/CursoAprendizajeProfundo/blob/master/notebooks/3a_rnn_class_last_step.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Reconocimiento de acciones con CNN + RNN

#### <div style="text-align: right"> Berenice Montalvo Lezama </div>

En este ejemplo veremos como realizar reconocimiento de acciones sobre [UCF11](https://www.crcv.ucf.edu/data/UCF_YouTube_Action.php) un conjunto de referencia en analisis de video recolectado por el [Center for Research in Computer Vision](https://www.crcv.ucf.edu/) de la Universidad de Central del Florida. UCF11 es un cunjunto multiclase con 1600 videos en 11 diferentes de acciones humanas. 

La arquitectura que se presenta en este ejemplo esta compuesta por una red neuronal convolucional y una recurrente. Para otorgar la clasificación de un video se toma la última salida de la red recurrente.


![UCF11](../figs/UCF11.jpg)



## 1 Carga de datos

### 1.1 Importando bibliotecas

In [1]:
try:
  %tensorflow_version 2.x
except Exception:
  pass

import os
from datetime import datetime

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, GRU, LSTM
from tensorflow.keras import Model

tf.random.set_seed(2019)

### 1.2 Descarga de los datos

In [2]:
tf.keras.utils.get_file('ucf11_frames_reps_rn50v2_pca_std.tfrecords',
    'https://cloud.xibalba.com.mx/s/PFZWERd9MdHoLpz/download',
    cache_subdir='datasets/ucf11')
!ls ~/.keras/datasets/ucf11

ucf11_frames_reps_rn50v2_pca_std.tfrecords


### 1.3 Tuberia de datos

In [3]:
# se crea un diccionario para la descripción de los ejemplos
_feat_desc = {
  'x': tf.io.VarLenFeature(tf.float32),
  'y': tf.io.FixedLenFeature([], tf.int64),
  'name': tf.io.FixedLenFeature([], tf.string)
}

def _parse_function(example_proto):
  se = tf.io.parse_single_example(example_proto, _feat_desc)
  x = tf.reshape(tf.sparse.to_dense(se['x']), (50, -1))
  y = se['y']
  name = se['name']
  return x, y, name

# ruta de los TFRecords con las representaciones espaciales 
ds_path = '~/.keras/datasets/ucf11/ucf11_frames_reps_rn50v2_pca_std.tfrecords'
ds_path = os.path.expanduser(ds_path)
shuffle_size = 1593
batch_size = 64

# se crea un TFRecordDataset para los TFRecords con las representaciones espaciales 
train_ds = tf.data.TFRecordDataset(ds_path)
# se obtienen las representaciones espaciales de los ejemplos
train_ds = train_ds.map(_parse_function)
# se barajea el conjunto
train_ds = train_ds.shuffle(shuffle_size)
# se contruyen los lotes 
train_ds = train_ds.batch(batch_size)

for x, y_true, name in train_ds.take(1):
    print(f'x.shape={x.shape}')
    print('x[0, :5, :5]=')
    print(x[0, :5, :5].numpy().flatten())
    print(f'y_true.shape={y_true.shape}')
    print(f'y_true[0]={y_true[0].numpy()}')
    print(f'name.shape={name.shape}')
    print(f'name[0]={name[0].numpy()}')

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
x.shape=(64, 50, 512)
x[0, :5, :5]=
[-0.36228782 -1.3095968   1.1650468   0.59512883  1.4917995  -0.36076042
 -1.3810619   1.2208297   0.62461144  1.6224496  -0.35992855 -1.377344
  1.2155812   0.618533    1.6204273  -0.3592346  -1.3868256   1.2387475
  0.624294    1.6373067  -0.36634874 -1.3755231   1.238343    0.60989845
  1.6211106 ]
y_true.shape=(64,)
y_true[0]=3
name.shape=(64,)
name[0]=b'golf_swing/v_golf_03/v_golf_03_04'


## 2 Definición del modelo

![Arquitectura](../figs/rnn_last_step.png)

In [4]:
class CRNNLS(tf.keras.Model):

  def __init__(self):
    super(CRNNLS, self).__init__()
    self.rl = GRU(32, name='rl')
    self.fc = Dense(11, activation='softmax', name='fc')

  def call(self, x):
    # (N, 50, 512) =>
    # (N, 32)
    x = self.rl(x)
    # (N, 32) =>
    # (N, 11)
    x = self.fc(x)
    return x

## 3 Entrenamiento

### 3.1 Función de pérdida y optimizador

In [5]:
# entropía cruzada binaria
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
# gradiente descendente
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)

### 3.2 Métricas: pérdida y exactitud

In [6]:
# historiales a nivel época
loss_epoch = tf.keras.metrics.Mean()
acc_epoch = tf.keras.metrics.SparseCategoricalAccuracy()

# historiales a nivel entrenamiento
loss_history = []
acc_history = []

### 3.3 Ciclo de entrenamiento

In [7]:
# instanciamos el modelo
model = CRNNLS()

model_name = datetime.now().strftime('%y%m%d-%H%M%S')
train_dir = f'logs/{model_name}/train'
train_writer = tf.summary.create_file_writer(train_dir)
print(f"Trainig model {model_name}")


for epoch in range(50):
    for step, (x, y_true, _) in enumerate(train_ds):

        with tf.GradientTape() as tape:
            
            y_pred = model(x)
            loss = loss_fn(y_true, y_pred)

        gradients = tape.gradient(loss, model.trainable_weights)
        optimizer.apply_gradients(zip(gradients, model.trainable_weights))
        
        loss_epoch(loss)
        acc_epoch(y_true, y_pred)

    loss_val = loss_epoch.result().numpy() * 100
    acc_val = acc_epoch.result().numpy() * 100
    loss_epoch.reset_states()
    acc_epoch.reset_states()
    
    with train_writer.as_default():
        tf.summary.scalar('loss', loss_val, epoch)
        tf.summary.scalar('acc', acc_val, epoch)
    
    print(f'{epoch:3d} loss={loss_val:6.2f}, acc={acc_val:6.2f}')

Trainig model 190923-212006
  0 loss=262.90, acc=  9.72
  1 loss=256.24, acc= 11.35
  2 loss=250.44, acc= 12.48
  3 loss=244.85, acc= 13.67
  4 loss=239.30, acc= 15.55
  5 loss=233.95, acc= 17.30
  6 loss=228.85, acc= 19.50
  7 loss=223.73, acc= 22.01
  8 loss=218.76, acc= 23.89
  9 loss=213.98, acc= 26.08
 10 loss=209.25, acc= 27.84
 11 loss=204.61, acc= 29.91
 12 loss=200.01, acc= 32.29
 13 loss=195.53, acc= 35.24
 14 loss=191.16, acc= 37.37
 15 loss=186.88, acc= 39.81
 16 loss=182.60, acc= 41.94
 17 loss=178.38, acc= 44.58
 18 loss=174.29, acc= 47.02
 19 loss=170.21, acc= 49.34
 20 loss=166.27, acc= 52.29
 21 loss=162.36, acc= 54.29
 22 loss=158.56, acc= 56.74
 23 loss=154.69, acc= 58.56
 24 loss=151.01, acc= 61.07
 25 loss=147.33, acc= 63.07
 26 loss=143.77, acc= 65.14
 27 loss=140.15, acc= 66.90
 28 loss=136.68, acc= 68.09
 29 loss=133.30, acc= 69.28
 30 loss=129.94, acc= 71.60
 31 loss=126.61, acc= 72.98
 32 loss=123.44, acc= 74.61
 33 loss=120.22, acc= 76.11
 34 loss=117.19, acc